In [ ]:
import geopandas as gpd


DATA_PATH = "/mnt/d/data/hackathon/"

# data processing

In [ ]:
import json
from pprint import pprint
import pandas as pd

postcode_df = pd.read_csv(os.path.join(DATA_PATH, "australian_postcodes.csv"))
postcode_df.loc[:, "locality"] = postcode_df["locality"].str.lower()
postcode_to_locality = dict(zip(postcode_df["locality"], postcode_df["postcode"]))
# print(postcode_to_locality)

custom_json = json.load(open(os.path.join(DATA_PATH, "LocalGovernmentArea_EPSG4326.json")))
custom_geojson = custom_json["LocalGovernmentArea"]

for feat in custom_geojson["features"]:
    # print(feat["properties"])
    _postcode = postcode_to_locality.get(feat["properties"]["lganame"].lower(), None)
    feat["properties"].update({"postcode": _postcode})
print(custom_geojson["features"][0]["properties"]["postcode"])
    

# plot

In [ ]:
import plotly.express as px

# Extract all LGA names from the GeoJSON features
locations = [feature["properties"]["postcode"] for feature in custom_geojson["features"] if feature["properties"]["postcode"]]

# Create a dummy color array (replace with your own data if available)
color = []
for _loc in locations:
    if _loc == 2077:
        _color = 100
    else:
        _color = 1
    color.append(_color)

fig = px.choropleth_map(
    geojson=custom_geojson,
    locations=locations,
    featureidkey="properties.postcode",
    color=color,
    center={"lat": -33.8688, "lon": 151.2093},  # Center on Sydney
    zoom=7,
    opacity=0.5,
)
fig.update_layout(width=600, height=600)
fig.show()